In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split


In [16]:
df=pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [18]:
df['Spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,Spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [19]:
X_train,X_test, y_train,y_test=train_test_split(df.Message, df.Spam,random_state=100)

In [20]:
#categorize the message column to numerical

from sklearn.feature_extraction.text import CountVectorizer
v=CountVectorizer()
X_train_count=v.fit_transform(X_train.values)
##print first three rows of vectorized message
X_train_count.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [21]:
#train the model
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_train_count,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
#test email with a sample

emails=[
    'Hey Diana. How are you doing? It\'s been long since we last talked. Wanna hangout sometime?',
    'Upto 20% discount on first 100 goods you buy. Amazing offers at our shop. Don\'t miss out!'
]

emails_count=v.transform(emails)
model.predict(emails_count)

array([0, 1], dtype=int64)

Model correctly predicts the different emails


In [23]:
X_test_count=v.transform(X_test)
model.score(X_test_count,y_test)

0.9863603732950467

## Sklearn pipeline

In [24]:
##prevents repetition of tasks
from sklearn.pipeline import Pipeline
clf=Pipeline([('Vectorizer', CountVectorizer()),
              ('nb', MultinomialNB())
             ])

In [25]:
clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('Vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('nb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [26]:
clf.score(X_test,y_test)

0.9863603732950467

In [27]:
clf.predict(emails)

array([0, 1], dtype=int64)